## 参数初始化
参数初始化对模型具有较大的影响，不同的初始化方式可能会导致截然不同的结果，所幸的是很多深度学习的先驱们已经帮我们探索了各种各样的初始化方式，所以我们只需要学会如何对模型的参数进行初始化的赋值即可。

PyTorch 的初始化方式并没有那么显然，如果你使用最原始的方式创建模型，那么你需要定义模型中的所有参数，当然这样你可以非常方便地定义每个变量的初始化方式，但是对于复杂的模型，这并不容易，而且我们推崇使用 Sequential 和 Module 来定义模型，所以这个时候我们就需要知道如何来自定义初始化方式
### 使用numpy来初始化

In [1]:
import numpy as np
import torch
from torch import nn

In [2]:
# 定义一个 Sequential 模型
net1 = nn.Sequential(
    nn.Linear(30, 40),
    nn.ReLU(),
    nn.Linear(40, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [3]:
# 访问第一层的参数
w1 = net1[0].weight
b1 = net1[0].bias

In [4]:
print(w1)

Parameter containing:
tensor([[-0.0211,  0.1012,  0.1771,  ...,  0.0519,  0.1788,  0.0409],
        [-0.0761,  0.1703,  0.0987,  ..., -0.0111, -0.0596, -0.0497],
        [-0.0518,  0.0783, -0.1807,  ...,  0.1638, -0.0300,  0.1513],
        ...,
        [ 0.0438,  0.1081, -0.0287,  ...,  0.1396, -0.1664, -0.1421],
        [-0.0674, -0.1531,  0.0836,  ..., -0.0984,  0.0671,  0.0563],
        [ 0.0875,  0.0579,  0.0702,  ...,  0.1555, -0.1627, -0.1609]],
       requires_grad=True)


注意，这是一个 Parameter，也就是一个特殊的 Variable，我们可以访问其 .data属性得到其中的数据，然后直接定义一个新的 Tensor 对其进行替换，我们可以使用 PyTorch 中的一些随机数据生成的方式，比如 torch.randn，如果要使用更多 PyTorch 中没有的随机化方式，可以使用 numpy

In [5]:
# 定义一个 Tensor 直接对其进行替换
net1[0].weight.data = torch.from_numpy(np.random.uniform(3, 5, size=(40, 30)))

In [6]:
print(net1[0].weight)

Parameter containing:
tensor([[4.8796, 3.6099, 3.0973,  ..., 4.5477, 4.6575, 3.7124],
        [4.6054, 3.8703, 3.5973,  ..., 4.0487, 4.8876, 4.7435],
        [3.6083, 4.2068, 3.2687,  ..., 4.6319, 3.5187, 4.1661],
        ...,
        [4.8797, 4.3885, 3.5457,  ..., 4.6154, 3.6522, 4.1037],
        [4.1442, 3.8577, 3.4044,  ..., 4.3633, 4.0764, 4.6938],
        [3.9548, 3.6856, 3.3853,  ..., 3.2122, 3.1545, 4.0532]],
       dtype=torch.float64, requires_grad=True)


可以看到这个参数的值已经被改变了，也就是说已经被定义成了我们需要的初始化方式，如果模型中某一层需要我们手动去修改，那么我们可以直接用这种方式去访问，但是更多的时候是模型中相同类型的层都需要初始化成相同的方式，这个时候一种更高效的方式是使用循环去访问，比如

In [7]:
for layer in net1:
    if isinstance(layer, nn.Linear): # 判断是否是线性层
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 
        # 定义为均值为 0，方差为 0.5 的正态分布


对于 Module 的参数初始化，其实也非常简单，如果想对其中的某层进行初始化，可以直接像 Sequential 一样对其 Tensor 进行重新定义，其唯一不同的地方在于，如果要用循环的方式访问，需要介绍两个属性，children 和 modules，下面我们举例来说明

In [8]:
class sim_net(nn.Module):
    def __init__(self):
        super(sim_net, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(30, 40),
            nn.ReLU()
        )
        
        self.l1[0].weight.data = torch.randn(40, 30) # 直接对某一层初始化
        
        self.l2 = nn.Sequential(
            nn.Linear(40, 50),
            nn.ReLU()
        )
        
        self.l3 = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.l1(x)
        x =self.l2(x)
        x = self.l3(x)
        return x 

In [9]:
net2 = sim_net()

In [10]:
# 访问 children
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)


In [11]:
# 访问 modules
for i in net2.modules():
    print(i)

sim_net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40, bias=True)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50, bias=True)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10, bias=True)
    (1): ReLU()
  )
)
Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Linear(in_features=30, out_features=40, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Linear(in_features=40, out_features=50, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)
Linear(in_features=50, out_features=10, bias=True)
ReLU()


children 只会访问到模型定义中的第一层，因为上面的模型中定义了三个 Sequential，所以只会访问到三个 Sequential，而 modules 会访问到最后的结构，比如上面的例子，modules 不仅访问到了 Sequential，也访问到了 Sequential 里面，这就对我们做初始化非常方便，比如

In [12]:
for layer in net2.modules():
    if isinstance(layer, nn.Linear):
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape))

### torch.nn.init
因为 PyTorch 灵活的特性，我们可以直接对 Tensor 进行操作从而初始化，PyTorch 也提供了初始化的函数帮助我们快速初始化，就是 torch.nn.init，其操作层面仍然在 Tensor 上，下面我们举例说明

In [13]:
from torch.nn import init
print(net1[0].weight)

Parameter containing:
tensor([[ 0.0196,  0.2776, -0.5477,  ...,  0.1330,  0.0029, -0.2583],
        [-0.0313, -0.9351,  0.2399,  ..., -0.4000, -0.0873, -0.8263],
        [ 0.1240,  0.2727, -0.6785,  ..., -0.7834,  0.7353,  0.7300],
        ...,
        [ 0.6228, -0.7604,  0.1134,  ..., -0.0706, -0.4829,  0.2401],
        [-0.3667,  0.3828,  0.0548,  ..., -0.5521,  0.7018,  0.4368],
        [-0.0350,  0.4979, -0.7052,  ..., -0.1376,  0.0971,  0.1718]],
       dtype=torch.float64, requires_grad=True)


In [14]:
init.xavier_uniform_(net1[0].weight)

Parameter containing:
tensor([[-0.1976, -0.0231, -0.0542,  ...,  0.2501, -0.0205, -0.2575],
        [ 0.1259, -0.1730, -0.0260,  ...,  0.0000, -0.0749, -0.1068],
        [ 0.2277, -0.2720, -0.1421,  ..., -0.2412, -0.2642, -0.1594],
        ...,
        [ 0.2856,  0.1766,  0.2797,  ...,  0.2496,  0.2526,  0.1163],
        [-0.0114,  0.2059,  0.2649,  ..., -0.1354, -0.2818,  0.0931],
        [-0.2107, -0.0523,  0.2117,  ...,  0.0574,  0.1098,  0.2413]],
       dtype=torch.float64, requires_grad=True)

torch.nn.init 为我们提供了更多的内置初始化方式，避免了我们重复去实现一些相同的操作

上面讲了两种初始化方式，其实它们的本质都是一样的，就是去修改某一层参数的实际值，而 torch.nn.init 提供了更多成熟的深度学习相关的初始化方式，非常方便